<a href="https://colab.research.google.com/github/AAkiraYamazaki/PollenCounter-YOLO/blob/main/colab/PollenCounter_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌼 PollenCounter-YOLOv5 — Colab Notebook

(2026/02/21 updated)


---



 This notebook enables automated quantification of strawberry pollen reproductive traits using deep learning-based object detection (YOLOv5).

It supports two tasks:  
	•	Pollen germination  
(classification: germinated / non-germinated)  
	•	Pollen viability  
(classification: viable / non-viable)   




---


This notebook will:   
	1.	Setup working directory  
	2.	Install YOLOv5 and PollenCounter   
	3.	Allow you to upload microscope images (or use Google Drive images)  
	4.	Automatically detect and classify pollen grains  
	5.	Export detection results


No local installation is required.



---




### Funding:
 This work was supported by the Research and Implementation Promotion Program through Open Innovation Grants (JPJ011937) from the Bio-oriented Technology Research Advancement Institution (BRAIN), Japan.



## 0) Select task
This Colab notebook supports both pollen germination and viability models. Please run the first cell and then select the desired task.


Select which model to use:  
	•	"germination" for pollen germination rate  
	•	"viability" for pollen viability

The selected model will automatically determine which weight file is used.


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display

# Map task -> weight file
WEIGHT_MAP = {
    "germination": "yolov5/weights/best_germination_v3.pt",
    "viability":   "yolov5/weights/best_viable_v5.pt",
}

# Dropdown
task_widget = widgets.Dropdown(
    options=list(WEIGHT_MAP.keys()),
    value="germination",
    description="Task:",
)
display(task_widget)

# Initialize (important!)
WEIGHT_PATH = WEIGHT_MAP[task_widget.value]
os.environ["WEIGHT_PATH"] = WEIGHT_PATH
print("Initialized WEIGHT_PATH =", WEIGHT_PATH)

def on_task_change(change):
    global WEIGHT_PATH
    if change["name"] == "value":
        task = change["new"]
        WEIGHT_PATH = WEIGHT_MAP[task]          # Python variable (string)
        os.environ["WEIGHT_PATH"] = WEIGHT_PATH # For !shell commands
        print(f"Selected task: {task}")
        print(f"WEIGHT_PATH set to: {WEIGHT_PATH}")

task_widget.observe(on_task_change, names="value")

## 1) Setup working directory
This step creates a temporary workspace in Colab where all files will be stored.

In [ ]:
WORKING_DIR = '/content/tmp'
!mkdir $WORKING_DIR
%cd $WORKING_DIR

## 2) Clone YOLOv5 and PollenCounter-YOLO repository

This step:  
	•	Downloads YOLOv5 framework  
	•	Installs required dependencies  
	•	Clones the PollenCounter-YOLO repository   

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5/
!pip install -qr requirements.txt
!git clone https://github.com/AAkiraYamazaki/PollenCounter-YOLO.git
!cp PollenCounter-YOLO/PollenCounter-YOLO.py .
!cp -r PollenCounter-YOLO/weights .
%cd ../

## 3) Provide input images
You can choose one of two options:  
### Option A — Upload Images Directly

Upload .jpg or .png microscope images directly from your computer.

Best for:  
	•	Small number of images.  
	•	Quick testing.

### Option B — Use Google Drive

If you have many images:  
	1.	Place images inside:   
       **MyDrive/PollenImages**  
	2.	Mount Google Drive.  
  3.	Set INPUT_DIR to the correct path.    


## 3) Option A. Upload Images Directly
Upload .jpg microscope images directly from your computer.

In [ ]:
# Option A: upload files manually (no Google Drive required)
from google.colab import files
uploaded = files.upload()

# Put uploaded files into a folder
import os, shutil
INPUT_DIR = "/content/work/input_images"
os.makedirs(INPUT_DIR, exist_ok=True)

for fn in uploaded.keys():
    shutil.move(fn, os.path.join(INPUT_DIR, fn))

print("Input images:", os.listdir(INPUT_DIR)[:10])

## 3) Option B. Use Google Drive
Place your images inside:
MyDrive/PollenImages


#### 3) Option B-1. Place image inside your Google Drive

Create a directory named **PollenImages** directly under your **MyDrive** in Google Drive, and upload your microscope images into this folder.


Create the following structure in your Google Drive:

MyDrive/  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── PollenImages/  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── sample1.jpg  
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   sample2.jpg  
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   sample3.jpg
   
If you would like to organize images by cultivar, sampling date, or experimental condition, you may create **one additional subdirectory level** inside PollenImages.

For example:

MyDrive/  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── PollenImages/  
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── folder_Kaorino/   
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── sample1.jpg    
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── folder_Yumenoka/    
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   └── sample2.jpg  
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   └── sample3.jpg


  ⚠️ Only **one level of subdirectories** is recommended for optimal compatibility with the detection script.




#### 3) Option B-2. Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### 3) Option B-3. Set Input directory

In [ ]:
INPUT_DIR = "/content/drive/MyDrive/PollenImages"

import os

if os.path.exists(INPUT_DIR):
    print("Input directory found.")
else:
    print("Input directory NOT found. Please check the path.")

## 4) Run detection

The detection script will:  
	•	Detect each pollen grains  
	•	Classify it according to the selected model.   
	•	Save results inside:  yolov5/runs/




In [ ]:
# Run detection

!python yolov5/PollenCounter-YOLO.py \
  --source $INPUT_DIR \
  --weights $WEIGHT_PATH \
  --save-txt

## 5) Download results
Detection results will be compressed into a .zip file for download.

The output folder includes:  
  •	Result summary file (.csv)  
	•	Annotated images  
	•	Detection label files (.txt)  
  

In [ ]:
import os
from google.colab import files

zip_name = "results.zip"
if os.path.exists(zip_name):
    os.remove(zip_name)
# Zip results
!zip -r results.zip yolov5/runs/
# Download
files.download(zip_name)

## Citation

If you use this model in your research, please cite:

 Akira Yamazaki, Natsuki Uematsu, Taro Sano, Yoshito Tatsumi, Keiji Mine, Hiroshi Asao, Haruyuki Atsumi, Yuhei Ueda, Ginyu Inaba, Kyosuke Kawabe, Kenichiro Yasuba, Yuichi Yoshida. Deep learning-based phenotyping of strawberry pollen reproductive capacity robust to imaging conditions enables multi-site analysis of environmental influences. Under Review.

